In [1]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html

import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_style('whitegrid')
#%matplotlib inline

from pandas_datareader import data
from datetime import datetime
nan=np.nan

In [2]:
def WACC_func (sticker,DBGmode):
    # WACC Calculator for a given stock
    
    # Cost of Debt Calculation not tax exempted
    
    # Read longTermDebt
    dframe_Balance_Sheet = si.get_balance_sheet(sticker)
    longTermDebt_col = dframe_Balance_Sheet.loc['longTermDebt',:]
    longTermDebt = longTermDebt_col[0] / 1000000 #convert to millions
    shortTermDebt = 0
    
    # Read interestExpense, incomeTaxExpense, incomeBeforeTax
    dframe_Income_Statement = si.get_income_statement(sticker)
    interestExpense_col = dframe_Income_Statement.loc['interestExpense',:]
    interestExpense = abs(interestExpense_col[0]) / 1000000 #convert to millions
    incomeTaxExpense_col = dframe_Income_Statement.loc['incomeTaxExpense', :]
    incomeTaxExpense = abs(incomeTaxExpense_col[0]) / 1000000 #convert to millions
    incomeBeforeTax_col = dframe_Income_Statement.loc['incomeBeforeTax', :]
    incomeBeforeTax = abs(incomeBeforeTax_col[0]) / 1000000 #convert to millions

    CostOfDebt_PreTax = interestExpense/longTermDebt
    
    # Effective Tax Rate Calculation
    EffTaxRate = incomeTaxExpense/incomeBeforeTax
    
    # Cost of Debt * (1-T) Tax Exmpted
    CostOfDebt = CostOfDebt_PreTax * (1-EffTaxRate)
    
    # Cost of Equity Calculation
    
    # Read Risk Free Interest Rate - Treasury Yield 10 Years
    url_bond = 'https://finance.yahoo.com/bonds'
    dframe_list_Bond = pd.read_html(url_bond)
    TreasRate = dframe_list_Bond[0]
    RiskFreeRate = TreasRate.loc[2,'Last Price']/100
    
    # Read Beta
    dframe_Stats = si.get_stats(sticker)
    BetaVal = float(dframe_Stats.loc[0,'Value']) #data is in str, convert to float
    
    #MarketReturn = 8.5%
    MarketReturn = 0.085
    
    CostOfEquity = RiskFreeRate + (BetaVal * (MarketReturn - RiskFreeRate))
    
    # Weight of Debt and Equity Calculation
    
    TotalDebt = longTermDebt + shortTermDebt
    
    # Read Market Cap
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    MarketCap_tab = dframe_list_Summary[1]
    MarketCap_str = MarketCap_tab.loc[0,1]
    MarketCap = float(MarketCap_str[:-1])
    if MarketCap_str[-1:]=='B':    #check if Billion
        MarketCap = float(MarketCap_str[:-1])*1000
    if MarketCap_str[-1:]=='T':    #check if Trillion
        MarketCap = float(MarketCap_str[:-1])*1000*1000
    
    # Weight of Debt
    WeightOfDebt = TotalDebt / (TotalDebt + MarketCap)
    WeightOfEquity = MarketCap / (TotalDebt + MarketCap)
    
    # Weighted Average Cost of Capital (WACC) Calculation
    WACC = (CostOfDebt * WeightOfDebt) + (CostOfEquity * WeightOfEquity)
    
    if DBGmode:
        print ('LongTermDebt=', longTermDebt)
        print ('shortTermDebt=', shortTermDebt)
        print ('interestExpense=', interestExpense)
        print ('incomeTaxExpense=', incomeTaxExpense)
        print ('incomeBeforeTax=', incomeBeforeTax)
        print ('CostOfDebt_PreTax=', CostOfDebt_PreTax)
        print ('EffTaxRate=', EffTaxRate)
        print ('CostOfDebt=', CostOfDebt)
        print ('RiskFreeRate=', RiskFreeRate)
        print ('BetaVal=', BetaVal)
        print ('MarketReturn=', MarketReturn)
        print ('TotalDebt=', TotalDebt)
        print ('MarketCap=', MarketCap)
        print ('WeightOfDebt=', WeightOfDebt)
        print ('WeightOfEquity=', WeightOfEquity)
        print ('WACC=', WACC)
    return WACC

In [3]:
def stock_monte_carlo_func (start_price,days,mu,sigma):
    dt=1/days
    price = np.zeros(days)
    price[0] = start_price
    
    shock = np.zeros(days)
    drift = np.zeros(days)
    
    for x in range(1,days):
        shock[x] = np.random.normal(loc=mu*dt,scale=sigma*np.sqrt(dt))
        drift[x] = mu*dt
        price[x] = price[x-1] + (price[x-1] * (drift[x] + shock[x]))
        
    return price

In [4]:
def RiskVal_func (sticker_list, period):
    end = datetime.now()
    start = datetime(end.year-period, end.month, end.day)

    for stock in sticker_list:
        globals()[stock] = data.DataReader(stock,'yahoo',start,end)

    closing_df = data.DataReader(sticker_list,'yahoo',start,end)['Adj Close']
    tech_rets = closing_df.pct_change()
    rets = tech_rets.dropna()
    risk = 100*rets.quantile([0.05,0.01])

    risk_df = pd.DataFrame(risk.T)
    risk_df.index.names = ['Sticker']
    risk_df.rename({0.05:str(period)+'yr/95% WC Risk%', 0.01:str(period)+'yr/99% WC Risk%'}, axis=1, inplace=True)
    return risk_df

In [5]:
def GrahamInVal_func (sticker,DBGmode):
    #sticker = "CRM"   
    
    # Read EPS
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    Summary_tab = dframe_list_Summary[1]
    EPS_str = Summary_tab.loc[3,1]
    EPS = float(EPS_str)

    # Read P/E
    #PERatio_str = Summary_tab.loc[2,1]
    #PERatio_read = float(PERatio_str)
    PERatio_fixed = 7.0
    PERatio = PERatio_fixed

    # Read 5 Yr Growth Rate
    url_Analysis = 'https://finance.yahoo.com/quote/' + sticker + '/analysis?p=' + sticker
    dframe_list_Analysis = pd.read_html(url_Analysis)
    GrowthEst_tab = dframe_list_Analysis[5]
    GrowthEst_str = GrowthEst_tab.loc[4,sticker]
    GrowthEst = float(GrowthEst_str[:-1])
    GrowthScale = 1

    # Read AAACorpBond Yield
    url_AAACorpBond = 'https://ycharts.com/indicators/moodys_seasoned_aaa_corporate_bond_yield'
    dframe_list_AAACorpBond = pd.read_html(url_AAACorpBond)
    #dframe_AAACorpBond = dframe_list_AAACorpBond[0]
    #AAACorpBond_str = dframe_AAACorpBond.loc[0,'Unnamed: 1']
    #AAACorpBond = float(AAACorpBond_str[:-1])
    AAACorpBond = float(dframe_list_AAACorpBond[3][1][0][:-1])
    CorpBond = 4.4

    GrahamInVal = round((EPS * (PERatio + (GrowthEst * GrowthScale)) * CorpBond) / AAACorpBond,2)

    # Read Current Val
    Summary_tab = dframe_list_Summary[0]
    PrevClose_str = Summary_tab.loc[0,1]
    PrevClose = round(float(PrevClose_str),2)

    if PrevClose < GrahamInVal:
        Decision = 'BUY'
    else:
        Decision = 'SELL'

    MarginRaw = (PrevClose/GrahamInVal)-1
    Margin = round(MarginRaw*100,2)
    #Margin = '{percent:.2%}'.format(percent=MarginRaw)
    
    if DBGmode:
        print ('EPS=', EPS)
        print ('PERatio=', PERatio)
        print ('GrowthEst=', GrowthEst)
        print ('GrowthScale=', GrowthScale)
        print ('AAACorpBond=', AAACorpBond)
        print ('CorpBond=', CorpBond)
        print ('PrevClose=', PrevClose)
        print ('Margin=', Margin)
        print ('Decision=', Decision)
        print ('GrahamInVal=', GrahamInVal)

    return {'Sticker':sticker,
             'GrahamInVal($)':GrahamInVal,
             'PrevClose($)':PrevClose,
             'Decision':Decision,
             'Margin(%)':Margin,
             'EPS':EPS,
             'GrowthEst(%)':GrowthEst}

In [6]:
def EPSMult_func(eps,growth5yr,wacc,DBGmode):
    yreps=np.zeros(5)
    yreps[0]=eps
    for i in range(1,5):
        yreps[i]=yreps[i-1]*(1+(growth5yr/100))

    yrprice=np.zeros(5)
    yrprice[4]=yreps[4]*growth5yr*2
    for i in range(3,-1,-1):
        yrprice[i]=yrprice[i+1]/(1+wacc)
    
    if DBGmode:
        print ('yreps[0]=', yreps[0])
        print ('yreps[1]=', yreps[1])
        print ('yreps[2]=', yreps[2])
        print ('yreps[3]=', yreps[3])
        print ('yreps[4]=', yreps[4])
        print ('yrprice[0]=', yrprice[0])
        print ('yrprice[1]=', yrprice[1])
        print ('yrprice[2]=', yrprice[2])
        print ('yrprice[3]=', yrprice[3])
        print ('yrprice[4]=', yrprice[4])
            
    return yrprice[0]

In [7]:
def DCFInVal_func(ticker,GRAVG1LAST0,wacc,DBGmode):
    #Intrinsic Value using Discounted Cash Flow Method

    url_fcf = 'https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/free-cash-flow'

    a1_df = pd.read_html(url_fcf)[0]
    a1_df.set_index(a1_df.columns[0], inplace=True)
    a1_df.rename(columns={a1_df.columns[0]:"FCF"}, inplace=True)
    a1_df.loc[a1_df.index[0]+1] = nan
    a1_df.loc[a1_df.index[0]+2] = nan
    a1_df.loc[9999] = nan
    a1_df.sort_index(ascending=False, inplace=True)
    a1_df['Change']=nan
    a1_df['DisFCF']=nan

    for i in range(7,-1,-1):
        a1_df.loc[a1_df.index[i]][1] = (a1_df.loc[a1_df.index[i]][0]/a1_df.loc[a1_df.index[i+1]][0])-1
    #print(a1_df.loc[a1_df.index[i]][0])
    #print(a1_df.index[i])

    if GRAVG1LAST0:
        Change_avg=a1_df[['Change']].mean(axis=0)[0]
    else:
        Change_avg=a1_df.loc[a1_df.index[3]][1]

    for i in range(2,0,-1):
        a1_df.loc[a1_df.index[i]][1] = Change_avg
        a1_df.loc[a1_df.index[i]][0] = a1_df.loc[a1_df.index[i+1]][0]*(1+a1_df.loc[a1_df.index[i]][1])

    a1_df.loc[a1_df.index[0]][1] = 0.025 # Average economy growth between 2-3%
    a1_df.loc[a1_df.index[0]][0] = (a1_df.loc[a1_df.index[1]][0]*(1+a1_df.loc[a1_df.index[0]][1]))/(wacc-a1_df.loc[a1_df.index[0]][1])

    a1_df.loc[a1_df.index[2]][2] = a1_df.loc[a1_df.index[2]][0]/(1+wacc)**1
    a1_df.loc[a1_df.index[1]][2] = a1_df.loc[a1_df.index[1]][0]/(1+wacc)**2
    a1_df.loc[a1_df.index[0]][2] = a1_df.loc[a1_df.index[0]][0]/(1+wacc)**2

    DiscFCF_sum = a1_df.loc[a1_df.index[0]][2] + a1_df.loc[a1_df.index[1]][2] + a1_df.loc[a1_df.index[2]][2]
    Cash_sum = abs(si.get_balance_sheet(ticker).loc['cash',:][0]/1000000)
    Debt_sum = abs(si.get_balance_sheet(ticker).loc['longTermDebt',:][0]/1000000)
    EquityVal = DiscFCF_sum + Cash_sum - Debt_sum 

    a2_df=si.get_stats(ticker)
    a2_df.set_index(a2_df.columns[0], inplace=True)
    if a2_df.loc['Shares Outstanding 5',:][0][-1:]=='M':
        SO = float(a2_df.loc['Shares Outstanding 5',:][0][:-1])
    if a2_df.loc['Shares Outstanding 5',:][0][-1:]=='B':
        SO = float(a2_df.loc['Shares Outstanding 5',:][0][:-1])*1000

    DCFInVal = round(EquityVal / SO,2)

    if DBGmode:
        print ('GRAVG1LAST0=', GRAVG1LAST0)
        for i in range(0,8):
            print ("Year=%d; FCF=%f; Change=%f; DiscFCF=%f" %(a1_df.index[i], a1_df.loc[a1_df.index[i]][0], a1_df.loc[a1_df.index[i]][1], a1_df.loc[a1_df.index[i]][2]))
        print ('Change_avg=', Change_avg)
        print ('Sum of DFCF=', DiscFCF_sum)
        print ('Cash_sum =', Cash_sum)
        print ('Debt_sum=', Debt_sum)
        print ('EquityVal=', EquityVal)
        print ('Shares Outstanding=', SO)
        print ('DCFInVal=', DCFInVal)
        
            
    return DCFInVal